In [70]:
import torch
import torch.nn as torchNN

In [71]:
class LSTM(torchNN.Module):
    def __init__(self, layer, units, inputShape):
        super(LSTM, self).__init__()

        self.lstm = torchNN.LSTM(
            inputShape,
            units,
            layer,
            batch_first = True
        )

        self.layer = layer
        self.units = units
        self.inputShape = inputShape

        self.liner = torchNN.Linear(
            units,
            5
        )

    def forward(self, inputData, hidden):
        x, h = self.lstm(inputData, hidden)
        x = self.liner(x)

        return x, h

    def InitHidden(self, batchSize):
        weight = next(self.parameters()).data
        hidden = weight.new(self.layer, batchSize, self.units).zero_(), weight.new(self.layer, batchSize, self.units).zero_()

        return hidden


In [72]:
batchSize = 10
units = 10
layer = 1
inputShape = 3              # 3bit 编码
sequenceLength = 7          # 输入序列长度
model = LSTM(layer, units, inputShape)

In [73]:
inputData = torch.rand(batchSize, sequenceLength, inputShape)
y = torch.rand(10, 5)
hidden = model.InitHidden(batchSize)

In [74]:
output, h = model(inputData, hidden)
output = output.max(1).values

In [75]:
lossFunction = torchNN.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.3)

In [76]:
for i in range(5100):
    output, hidden = model(inputData, hidden)
    output = output.max(1).values
    loss = lossFunction(y, output)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss.item())

0.2382557988166809


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.